In [ ]:
from __future__ import print_function
import os,sys
import ROOT as rt
from larlite import larlite
from ROOT import larutil
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import lardly

In [ ]:
# LOAD THE FILE
HAS_MCTRACK = True

triplet_files = ["test_ub2.root"]
tree = rt.TChain("larmatchtriplet")
if HAS_MCTRACK:
    ioll = larlite.storage_manager(larlite.storage_manager.kREAD)
for f in triplet_files:
    tree.AddFile( f )
    if HAS_MCTRACK:
        ioll.add_in_filename( f.replace(".root","_larlite.root") )
nentries = tree.GetEntries()
if HAS_MCTRACK:
    ioll.open()
print("Num entries: ",nentries)

In [ ]:
# SETUP THE GEOMETRY

# Set the detector in larlite
#detid = larlite.geo.kICARUS
detid = larlite.geo.kMicroBooNE
#detid = larlite.geo.kSBND
larutil.LArUtilConfig.SetDetector(detid)

# Get geometry class
geo = larlite.larutil.Geometry.GetME()

# Get detector outlines from lardly
from lardly.detectors.getdetectoroutlines import DetectorOutlineFromLarlite
detoutline = DetectorOutlineFromLarlite(detid)
detlines = detoutline.getlines()

In [ ]:
# DEFINE PARTICLE ID COLORS
particle_id_color = {0:(0,0,0),      # no label
                     1:(255,125,50), # Cosmic
                     2:(0,0,0),      # BNB
                     3:(255,0,0),    # electron
                     4:(0,255,0),    # gamma
                     5:(0,125,125),  # pi0
                     6:(155,0,155),  # Muon
                     7:(255,255,0),  # Kaon
                     8:(255,165,0),  # pion                     
                     9:(0,0,255)}    # proton

particle_id_name = {0:"nolabel",  # no label
                    1:"delta",    # no label
                    2:"nolabel",  # no label
                    3:"electron", # electron
                    4:"gamma",    # gamma
                    5:"pi0",  # pi0
                    6:"muon", # Muon
                    7:"kaon", # Kaon
                    8:"pion", # proton
                    9:"proton"}   # pion

In [ ]:
# Get entry data
ENTRY = 9
tree.GetEntry(ENTRY)
ioll.go_to(ENTRY)

spacepoint_arrays = []
for imatchdata in range(tree.triplet_v.size()):
    matchdata = tree.triplet_v.at(imatchdata)
    ntriplets = matchdata._triplet_v.size() # needs function
    if ntriplets==0:
        continue
    icryo = matchdata._cryoid # needs function
    itpc  = matchdata._tpcid # needs function
    
    spdata = matchdata.make_spacepoint_charge_array()
    print("keys: ",spdata.keys())
    print("spacepoint_t:",spdata['spacepoint_t'].shape)
    print("truetriplet_t:",spdata['truetriplet_t'].shape)
    spacepoint_arrays.append( (icryo,itpc,spdata) )
    
ev_mctrack = ioll.get_data(larlite.data.kMCTrack,"mcreco")

In [ ]:
# PLOT SPACE POINT CHARGE

sp_plots = []
for (icryo,itpc,spdata) in spacepoint_arrays:
    pos = spdata['spacepoint_t']
    q = np.clip( np.average( pos[:,3:], axis=1 ), 0, 100 )
    plot = go.Scatter3d( x=pos[:,0], y=pos[:,1], z=pos[:,2], 
                        mode="markers", name="(%d,%d)"%(icryo,itpc), 
                        marker={"size":0.5,"opacity":0.1,"color":q,"colorscale":"Viridis"} )  
    sp_plots.append(plot)

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='DETECTOR TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 2, "y": 1, "z": 2},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=sp_plots+detlines, layout=layout)
fig.show()
 

In [ ]:
# PLOT TRUTH SPACEPOINTS

sp_plots = []
for (icryo,itpc,spdata) in spacepoint_arrays:
    pos = spdata['spacepoint_t']
    truth = spdata['truetriplet_t']

        
    
    plot = go.Scatter3d( x=pos[:,0], y=pos[:,1], z=pos[:,2], 
                        mode="markers", name="(%d,%d)"%(icryo,itpc), 
                        marker={"size":0.5,"opacity":0.1,"color":truth,"colorscale":"Viridis"} )  
    sp_plots.append(plot)

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='DETECTOR TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 2},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=sp_plots+detlines, layout=layout)
fig.show()

In [ ]:
# PLOT SEGMENT SPACEPOINTS
TRUTH_ONLY = True
PLOT_MCTRACKS = True

sp_plots = []
for (icryo,itpc,spdata) in spacepoint_arrays:
    truth = spdata['truetriplet_t']
    if TRUTH_ONLY:
        pos = spdata['spacepoint_t'][truth==1,:]
        pdg = spdata['segment_t'][truth==1]
    else:
        pos = spdata['spacepoint_t']
        pdg = spdata['segment_t']

    
    npts = pos.shape[0]
    unique = np.unique(pdg)
    color = np.zeros( (npts,3) )
    print("unique ids: ",unique)
    for pdgid in unique:
        #if pdgid not in [9]:
        #    continue
        part_pos = pos[pdg==pdgid,:]
        partcolor = "rgb(%d,%d,%d)"%(particle_id_color[pdgid][0],particle_id_color[pdgid][1],particle_id_color[pdgid][2])
        plot = go.Scatter3d( x=part_pos[:,0], y=part_pos[:,1], z=part_pos[:,2], 
                            mode="markers", name="%d"%(pdgid), 
                            marker={"size":1.0,"opacity":1.0,"color":partcolor} )  
        sp_plots.append(plot)
    
if PLOT_MCTRACKS:
    traces_v =   lardly.data.visualize_larlite_event_mctrack( ev_mctrack,origin=1,do_sce_correction=True)
    sp_plots += traces_v


axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='DETECTOR TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 2},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=sp_plots+detlines, layout=layout)
fig.show()